# Calculate the percentage of anomaly for a specific basin
#### Jose Valles (jose.valles.leon@gmail.com)

### Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('classic')
import numpy as np
import calendar

sns.set()

from IPython.display import HTML

In [2]:
# Define the basin level 2 code 
codcuenca_n2 = 23
# Define the variables
model_variable = 'Escorrentia'

In [3]:
basin_level3 = pd.read_csv(f'../output_modelo/cuenca_nivel3.csv',usecols=lambda col: col.startswith(str(codcuenca_n2)))
basin_level2 = pd.read_csv(f'../output_modelo/cuenca_nivel2.csv',usecols=lambda col: col.startswith(str(codcuenca_n2)))

In [4]:
def importmodelvariable(codcuenca_n2,model_variable):
    df = pd.read_csv(f'../output_modelo/{model_variable}.csv',usecols=lambda col: col.startswith((str(codcuenca_n2),'-1')))
    df = df.rename(columns={'-1': 'year','-1.1':'month'})
    df['date'] = pd.to_datetime(dict(year=df['year'],month=df['month'],day=1))
    df = df.set_index('date')
    df['days_in_month'] = df.index.days_in_month
    return df

def convertRunoff2Discharge(df_runoff):
    df_runoff_selected = df_runoff.drop(['year','month','days_in_month'],axis=1)
    df_discharge = pd.DataFrame(df_runoff_selected.values*1000*basin_level3.values,columns=df_runoff_selected.columns)
    df_discharge['days_in_month'] = df_runoff['days_in_month'].values
    df_discharge = df_discharge.loc[:, df_discharge.columns != 'days_in_month'].divide(df_discharge["days_in_month"]*24*3600, axis="index")
    df_discharge['date'] = df_runoff.index.values
    df_discharge = df_discharge.set_index('date')
    df_discharge['year'] = df_runoff['year'].values
    df_discharge['month'] = df_runoff['month'].values
    return df_discharge

In [5]:
hydrological_variable = ['Escorrentia','Pmedias','ETR','HumedadSuelo']

for hydro in hydrological_variable:
    locals()[hydro] = importmodelvariable(codcuenca_n2,hydro)

### Importar datos de modelos

Importar datos de precipitación, escorrentia, humedad suelo y Evapotranspiración (potencial y actual)

In [6]:
# Escorrentia
RUNOFF = pd.read_csv('../output_modelo/Escorrentia.csv')
RUNOFF = RUNOFF.rename(columns={'-1': 'year','-1.1':'month'})
RUNOFF['date'] = pd.to_datetime(dict(year=RUNOFF['year'],month=RUNOFF['month'],day=1))
RUNOFF = RUNOFF.set_index('date')
RUNOFF['days_in_month'] = RUNOFF.index.days_in_month

# Precipitación
PRECIP = pd.read_csv('../output_modelo/Pmedias.csv')
PRECIP = PRECIP.rename(columns={'-1': 'year','-1.1':'month'})
PRECIP['date'] = pd.to_datetime(dict(year=PRECIP['year'],month=PRECIP['month'],day=1))

# Evapotranspiración Real
ETR = pd.read_csv('../output_modelo/ETR.csv')
ETR = ETR.rename(columns={'-1': 'year','-1.1':'month'})
ETR['date'] = pd.to_datetime(dict(year=ETR['year'],month=ETR['month'],day=1))

# Humedad Suelo 
SM = pd.read_csv('../output_modelo/HumedadSuelo.csv')
SM = SM.rename(columns={'-1': 'year','-1.1':'month'})
SM['date'] = pd.to_datetime(dict(year=SM['year'],month=SM['month'],day=1))


Importar codigos de cuenca nivel 2 y 3

In [7]:
BASIN_LEVEL3 = pd.read_csv(f'../output_modelo/cuenca_nivel3.csv',index_col="Codigo")
BASIN_LEVEL2 = pd.read_csv(f'../output_modelo/cuenca_nivel2.csv',index_col="Codigo")

In [8]:
HTML(RUNOFF.tail(6).to_html(index=False))

year,month,101,102,103,105,106,107,108,109,110,111,112,114,115,116,117,119,120,123,125,128,130,131,132,133,134,135,136,137,138,139,140,142,146,148,150,155,158,160,163,165,167,168,170,171,172,173,174,175,176,177,178,179,180,183,186,189,190,193,196,199,201,204,208,210,211,212,213,214,215,216,217,220,221,222,223,224,225,226,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,260,262,264,266,268,270,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,293,294,295,297,298,300,301,305,306,310,312,315,316,318,320,325,330,333,334,335,338,406,407,410,411,412,413,414,415,416,417,418,419,420,422,424,426,428,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,455,457,458,459,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,522,524,526,530,532,534,536,538,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,575,580,581,582,583,584,585,586,587,588,589,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,618,620,625,630,635,640,645,650,651,652,653,654,655,656,657,658,660,661,662,666,670,673,677,680,683,687,days_in_month
2022,12,3.88770,3.89020,2.71840,2.49150,2.04800,2.39700,3.93120,2.53360,2.23180,2.247000,2.303000,2.378000,2.73380,3.01430,2.97750,4.62700,6.32330,6.52480,6.18950,5.78980,4.96420,5.29880,6.281000,6.397000,7.0079,7.31650,7.69270,7.57150,7.13410,7.09920,6.38530,7.07820,7.21560,6.87950,7.09450,7.19250,7.12800,6.77890,6.87280,6.40200,5.81330,4.42790,4.38670,5.2071,1.87810,1.75590,3.9880,3.68460,1.74320,1.58900,2.75350,3.01770,1.421800,1.107900,1.37060,0.053863,0.000953,0.001815,0.000153,1.123200e-04,2.101200e-05,2.853200e-05,2.076200e-05,0.000480,1.076600e-04,3.959000e-05,3.155600e-05,2.629500e-05,3.299900e-05,2.975000e-05,2.098900e-05,2.118100e-05,2.993800e-05,3.271200e-05,4.083100e-05,3.923600e-05,3.905200e-05,3.828300e-05,0.000926,0.000918,0.000827,0.000907,0.000588,0.000449,9.597600e-05,0.000149,3.303400e-05,3.550100e-05,3.366500e-05,6.650400e-05,0.000352,0.000460,0.000266,8.294700e-05,0.000150,0.000146,1.007800e-04,0.087564,0.113720,0.103120,0.083226,0.193500,0.099278,0.623850,0.538240,0.602040,0.482390,0.358180,1.04890,0.819060,0.762750,0.80296,0.87064,0.674810,0.616900,0.712900,0.710470,0.522430,0.519310,0.652330,1.159600,1.39050,2.29340,2.504900,2.556500,2.33400,2.13180,2.694900,2.897600,3.22200,4.15910,4.408500,4.94550,4.973500,5.10970,5.80800,5.83390,5.21800,6.47050,6.88100,4.23340,2.70390,4.44460,4.29210,3.92070,3.68220,3.79750,3.81070,3.8256,3.75120,2.86530,3.11140,2.56140,2.73450,2.76130,3.11650,3.05780,2.04230,2.74660,1.76610,1.55860,1.35940,2.41750,1.71640,2.91430,2.51300,4.93820,2.8631,1.99220,1.59760,1.62300,1.81060,1.90650,2.44400,3.19100,3.34600,4.30670,3.79170,2.83810,3.66660,5.00400,6.6011,6.51410,6.65590,4.188100,4.099200,3.914200,3.893700,4.167500,4.31540,3.93430,3.48450,2.56780,4.355200,4.313400,4.151000,4.40590,4.76180,5.37780,6.33310,5.064100,5.920500,5.737500,6.226700,6.28230,6.186700,6.131500,5.640200,5.954200,5.319800,4.755900,4.050600,3.220500,5.175900,5.15350,3.049200,2.358300,2.630900,3.10620,2.67480,2.89520,2.85010,2.35230,2.450200,2.32050,3.100900,2.247100,2.013600,1.95540,2.18040,1.98100,1.50610,1.18920,0.480810,1.08370,0.054889,0.000935,0.310250,0.000892,0.095518,0.320860,1.12880,0.002872,0.277710,0.459920,2.10230,1.73400,0.126100,0.068388,0.82137,0.028704,0.011638,0.011436,0.007251,1.15180,1.19080,1.11870,0.735300,0.768490,1.11620,0.608100,0.564730,0.370800,0.274560,3.4867,1.22200,0.224290,0.000797,0.351440,0.175640,0.105390,0.000280,0.017021,0.014850,0.004953,1.028500e-04,0.000145,0.001100,0.000903,0.001003,0.001144,0.001093,0.000905,0.000692,0.000222,0.000282,5.606400e-05,6.528800e-05,3.567100e-05,7.209000e-05,0.019159,0.002445,0.056309,1.977900e-05,6.331000e-05,0.038324,31
2023,1,1.44460,1.44550,1.01010,0.92577,0.76100,0.89067,1.46070,0.94144,0.82929,0.834950,0.855760,0.883610,1.01580,1.12000,1.10640,1.719

### Convertir Escorrentia (mm/month) a Caudal (m3/s)

In [9]:
# Convertir datos de escorrentia a caudal
RUNOFF_SELECTION = RUNOFF.drop(['year','month','days_in_month'],axis=1)
DISCHARGE = pd.DataFrame(RUNOFF_SELECTION.values*1000*BASIN_LEVEL3.values,columns=RUNOFF_SELECTION.columns)
DISCHARGE['days_in_month'] = RUNOFF['days_in_month'].values
DISCHARGE = DISCHARGE.loc[:, DISCHARGE.columns != 'days_in_month'].divide(DISCHARGE["days_in_month"]*24*3600, axis="index")
# Definir columns de date y año
DISCHARGE['date'] = RUNOFF.index.values
DISCHARGE = DISCHARGE.set_index('date')
DISCHARGE['year'] = RUNOFF['year'].values
DISCHARGE['month'] = RUNOFF['month'].values

In [10]:
HTML(RUNOFF.tail(6).to_html(index=True))

,year,month,101,102,103,105,106,107,108,109,110,111,112,114,115,116,117,119,120,123,125,128,130,131,132,133,134,135,136,137,138,139,140,142,146,148,150,155,158,160,163,165,167,168,170,171,172,173,174,175,176,177,178,179,180,183,186,189,190,193,196,199,201,204,208,210,211,212,213,214,215,216,217,220,221,222,223,224,225,226,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,260,262,264,266,268,270,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,293,294,295,297,298,300,301,305,306,310,312,315,316,318,320,325,330,333,334,335,338,406,407,410,411,412,413,414,415,416,417,418,419,420,422,424,426,428,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,455,457,458,459,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,522,524,526,530,532,534,536,538,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,575,580,581,582,583,584,585,586,587,588,589,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,618,620,625,630,635,640,645,650,651,652,653,654,655,656,657,658,660,661,662,666,670,673,677,680,683,687,days_in_month
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-12-01,2022,12,3.88770,3.89020,2.71840,2.49150,2.04800,2.39700,3.93120,2.53360,2.23180,2.247000,2.303000,2.378000,2.73380,3.01430,2.97750,4.62700,6.32330,6.52480,6.18950,5.78980,4.96420,5.29880,6.281000,6.397000,7.0079,7.31650,7.69270,7.57150,7.13410,7.09920,6.38530,7.07820,7.21560,6.87950,7.09450,7.19250,7.12800,6.77890,6.87280,6.40200,5.81330,4.42790,4.38670,5.2071,1.87810,1.75590,3.9880,3.68460,1.74320,1.58900,2.75350,3.01770,1.421800,1.107900,1.37060,0.053863,0.000953,0.001815,0.000153,1.123200e-04,2.101200e-05,2.853200e-05,2.076200e-05,0.000480,1.076600e-04,3.959000e-05,3.155600e-05,2.629500e-05,3.299900e-05,2.975000e-05,2.098900e-05,2.118100e-05,2.993800e-05,3.271200e-05,4.083100e-05,3.923600e-05,3.905200e-05,3.828300e-05,0.000926,0.000918,0.000827,0.000907,0.000588,0.000449,9.597600e-05,0.000149,3.303400e-05,3.550100e-05,3.366500e-05,6.650400e-05,0.000352,0.000460,0.000266,8.294700e-05,0.000150,0.000146,1.007800e-04,0.087564,0.113720,0.103120,0.083226,0.193500,0.099278,0.623850,0.538240,0.602040,0.482390,0.358180,1.04890,0.819060,0.762750,0.80296,0.87064,0.674810,0.616900,0.712900,0.710470,0.522430,0.519310,0.652330,1.159600,1.39050,2.29340,2.504900,2.556500,2.33400,2.13180,2.694900,2.897600,3.22200,4.15910,4.408500,4.94550,4.973500,5.10970,5.80800,5.83390,5.21800,6.47050,6.88100,4.23340,2.70390,4.44460,4.29210,3.92070,3.68220,3.79750,3.81070,3.8256,3.75120,2.86530,3.11140,2.56140,2.73450,2.76130,3.11650,3.05780,2.04230,2.74660,1.76610,1.55860,1.35940,2.41750,1.71640,2.91430,2.51300,4.93820,2.8631,1.99220,1.59760,1.62300,1.81060,1.90650,2.44400,3.19100,3.34600,4.30670,3.79170,2.83810,3.66660,5.00400,6.6011,6.51410,6.65590,4.188100,4.099200,3.914200,3.893700,4.167500,4.31540,3.93430,3.48450,2.56780,4.355200,4.313400,4.151000,4.40590,4.76180,5.37780,6.33310,5.064100,5.920500,5.737500,6.226700,6.28230,6.186700,6.131500,5.640200,5.954200,5.319800,4.755900,4.050600,3.220500,5.175900,5.15350,3.049200,2.358300,2.630900,3.10620,2.67480,2.89520,2.85010,2.35230,2.450200,2.32050,3.100900,2.247100,2.013600,1.95540,2.18040,1.98100,1.50610,1.18920,0.480810,1.08370,0.054889,0.000935,0.310250,0.000892,0.095518,0.320860,1.12880,0.002872,0.277710,0.459920,2.10230,1.73400,0.126100,0.068388,0.82137,0.028704,0.011638,0.011436,0.007251,1.15180,1.19080,1.11870,0.735300,0.768490,1.11620,0.608100,0.564730,0.370800,0.274560,3.4867,1.22200,0.224290,0.000797,0.351440,0.175640,0.105390,0.000280,0.017021,0.014850,0.004953,1.028500e-04,0.000145,0.001100,0.000903

## Seleccionar periodo referencia

Seleccionar la variable de interes

In [11]:
year_start = 1981
year_end = 2010
# Caudal
SELECTED_REF_DISCHARGE= DISCHARGE[(DISCHARGE['year'] >= year_start) & (DISCHARGE['year'] <= year_end)]
# Escorrentia
SELECTED_REF_RUNOFF = RUNOFF[(RUNOFF['year'] >= year_start) & (RUNOFF['year'] <= year_end)]
# Precip
SELECTED_REF_PRECIP = PRECIP[(PRECIP['year'] >= year_start) & (PRECIP['year'] <= year_end)]

## Analisis Pais

#### Calculo anual

In [12]:
RUNOFF['avg_runoff'] = RUNOFF.iloc[:,2:-1].mean(axis=1)

In [13]:
sum_avg_runoff_year = RUNOFF.groupby(['year'])['avg_runoff'].sum()
sum_avg_runoff_year = sum_avg_runoff_year.to_frame()
sum_avg_runoff_month = RUNOFF.groupby(['year','month'])['avg_runoff'].sum()
sum_avg_runoff_month = sum_avg_runoff_month.to_frame()

In [14]:
display(sum_avg_runoff_month)

avg_runoff
year month            
1980 1        8.000802
     2       20.715013
     3       15.634677
     4       57.137607
     5       52.502548
...                ...
2023 1        0.878558
     2        0.337508
     3        4.850028
     4        2.683451
     5        1.419095

[521 rows x 1 columns]

In [15]:
sum_avg_runoff_month.to_clipboard()

## Calcula el porcentaje de anomalía por cuenca nivel 2

In [16]:
#--
month_analyis = 1
year_analysis = 2023

AVERAGE_PERCENTAGE = pd.DataFrame()
AVERAGE_PERCENTAGE['codigo'] = BASIN_LEVEL2.columns
AVERAGE_PERCENTAGE['discharge'] = np.nan
AVERAGE_PERCENTAGE['runoff'] = np.nan
AVERAGE_PERCENTAGE['precip'] = np.nan



for basin in BASIN_LEVEL2.columns:
    # filter the basin code bas
    filter_col = [col for col in DISCHARGE if col.startswith(str(basin))]
    # Select basins level 3 that belongs to level 2 basin
    # Discharge
    DISCHARGE_FILTER = DISCHARGE[filter_col]
    DISCHARGE_SELECTED = pd.DataFrame()
    DISCHARGE_SELECTED['year'] = DISCHARGE['year']
    DISCHARGE_SELECTED['month'] = DISCHARGE['month']    
    DISCHARGE_SELECTED['discharge'] = DISCHARGE_FILTER.sum(axis=1)
    DISCHARGE_REF = DISCHARGE_SELECTED[(DISCHARGE_SELECTED['year'] >= year_start) & (DISCHARGE_SELECTED['year'] <= year_end)]
    # Runoff
    RUNOFF_FILTER = RUNOFF[filter_col]
    RUNOFF_SELECTED = pd.DataFrame()
    RUNOFF_SELECTED['year'] = RUNOFF['year']
    RUNOFF_SELECTED['month'] = RUNOFF['month']    
    RUNOFF_SELECTED['runoff'] = RUNOFF_FILTER.mean(axis=1)
    RUNOFF_REF = RUNOFF_SELECTED[(RUNOFF_SELECTED['year'] >= year_start) & (RUNOFF_SELECTED['year'] <= year_end)]
    # Precip
    PRECIP_FILTER = PRECIP[filter_col]
    PRECIP_SELECTED = pd.DataFrame()
    PRECIP_SELECTED['year'] = PRECIP['year']
    PRECIP_SELECTED['month'] = PRECIP['month']    
    PRECIP_SELECTED['precip'] = PRECIP_FILTER.mean(axis=1)
    PRECIP_REF = PRECIP_SELECTED[(PRECIP_SELECTED['year'] >= year_start) & (PRECIP_SELECTED['year'] <= year_end)]
    # CALCULATE CLIMATOLOGICAL NORMALS
    # Discharge
    DISCHARGE_AVERAGE = DISCHARGE_REF.groupby(DISCHARGE_REF.month).mean()
    # Runoff
    RUNOFF_AVERAGE = RUNOFF_REF.groupby(RUNOFF_REF.month).mean()
    # Precip
    PRECIP_AVERAGE = PRECIP_REF.groupby(PRECIP_REF.month).mean()
    # Calculate the anomaly
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'discharge'] = (DISCHARGE_SELECTED.query('month == @month_analyis & year == @year_analysis')['discharge'].item() - DISCHARGE_AVERAGE.query('month == @month_analyis')['discharge'].item()) / DISCHARGE_AVERAGE.query('month == @month_analyis')['discharge'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'runoff'] = (RUNOFF_SELECTED.query('month == @month_analyis & year == @year_analysis')['runoff'].item()- RUNOFF_AVERAGE.query('month == @month_analyis')['runoff'].item()) / RUNOFF_AVERAGE.query('month == @month_analyis')['runoff'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'precip'] = (PRECIP_SELECTED.query('month == @month_analyis & year == @year_analysis')['precip'].item() - PRECIP_AVERAGE.query('month == @month_analyis')['precip'].item()) / PRECIP_AVERAGE.query('month == @month_analyis')['precip'].item()

In [17]:
HTML(AVERAGE_PERCENTAGE.head(6).to_html(index=False))

codigo,discharge,runoff,precip
10,-0.968134,-0.971191,-0.798776
11,-0.967974,-0.974503,-0.840918
12,-0.920807,-0.919725,-0.532791
13,-0.892612,-0.898608,-0.422263
14,-0.886355,-0.895127,-0.596705
15,-0.897101,-0.898773,-0.465855


In [18]:
AVERAGE_PERCENTAGE.to_clipboard(index=False)

In [19]:
#--
year_analysis = 2019

AVERAGE_PERCENTAGE = pd.DataFrame()
AVERAGE_PERCENTAGE['codigo'] = BASIN_LEVEL2.columns
AVERAGE_PERCENTAGE['runoff'] = np.nan
AVERAGE_PERCENTAGE['runoff_anomaly'] = np.nan
AVERAGE_PERCENTAGE['runoff_mean'] = np.nan



for basin in BASIN_LEVEL2.columns:
    # filter the basin code bas
    filter_col = [col for col in DISCHARGE if col.startswith(str(basin))]
    # Select basins level 3 that belongs to level 2 basin
    # Runoff
    RUNOFF_FILTER = RUNOFF[filter_col]
    RUNOFF_SELECTED = pd.DataFrame()
    RUNOFF_SELECTED['year'] = RUNOFF['year']
    RUNOFF_SELECTED['month'] = RUNOFF['month']    
    RUNOFF_SELECTED['runoff'] = RUNOFF_FILTER.mean(axis=1)
    RUNOFF_REF = RUNOFF_SELECTED[(RUNOFF_SELECTED['year'] >= year_start) & (RUNOFF_SELECTED['year'] <= year_end)]

    sum_avg_runoff_year = RUNOFF_SELECTED.groupby(['year'])['runoff'].sum()
    sum_avg_runoff_year = sum_avg_runoff_year.to_frame()

    # Runoff
    RUNOFF_AVERAGE = RUNOFF_REF.groupby(['year'])['runoff'].sum()
    RUNOFF_AVERAGE = RUNOFF_AVERAGE.to_frame()

    # Calculate the anomaly
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'runoff'] = sum_avg_runoff_year.query('year == @year_analysis')['runoff'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'runoff_anomaly'] = (sum_avg_runoff_year.query('year == @year_analysis')['runoff'].item() - RUNOFF_AVERAGE['runoff'].mean()) / RUNOFF_AVERAGE['runoff'].mean()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'runoff_mean'] = RUNOFF_AVERAGE['runoff'].mean()

In [20]:
AVERAGE_PERCENTAGE

,codigo,runoff,runoff_anomaly,runoff_mean
0,10,887.031000,0.588473,558.417338
1,11,798.323875,0.709706,466.936418
2,12,770.736250,0.513653,509.189507
3,13,771.396390,0.579753,488.301809
4,14,647.518550,0.764475,366.975182
5,15,675.509533,0.605171,420.833452
6,16,660.649440,0.850310,357.048019
7,17,763.750680,1.033690,375.549280
8,18,712.783350,1.489058,286.366750
9,19,334.223975,0.416084,236.019876


In [21]:
AVERAGE_PERCENTAGE.to_clipboard(index=False)